# Uploading dataset

In [ ]:
# Mount drive
#from google.colab import drive
#drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
from sklearn.model_selection import train_test_split
import os

# Set the path to your dataset
#dataset_path = '/content/Training set' # images
dataset_path = '/Users/macbookair/Desktop/541_model/input/Images' # images

# Get a list of all image files in the dataset
image_files = [os.path.join(dataset_path, f) for f in os.listdir(dataset_path) if f.endswith('.jpg')]

# Split the image files into training and testing sets
train_files, test_files = train_test_split(image_files, test_size=0.2, random_state=42, shuffle=True)

# Print the number of images in each set
print(f'Training set size: {len(train_files)}')
print(f'Testing set size: {len(test_files)}')

Training set size: 2205
Testing set size: 552


In [ ]:
#!pip install split-folders
import splitfolders


input_folder = '/Users/macbookair/Desktop/541_model/input'


# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, (.8, .2).
# Train, val, test
splitfolders.ratio(input_folder,
                   output="Images_for_train_test",
                   seed=1337,
                   ratio=(.8, .2),
                   group_prefix=None)

Copying files: 2757 files [00:17, 156.57 files/s]


# Downloading pretrained tensor flow model for detection of cards


In [ ]:
from pandas.core.dtypes.common import is_array_like
import tensorflow as tf
import cv2
import numpy as np
import urllib.request

# Download the pre-trained SSD model
model_url = 'http://download.tensorflow.org/models/object_detection/ssd_mobilenet_v1_coco_2018_01_28.tar.gz'
model_file = 'ssd_mobilenet_v1_coco_2018_01_28/frozen_inference_graph.pb'

urllib.request.urlretrieve(model_url, './model.tar.gz')
import tarfile
with tarfile.open('./model.tar.gz', 'r') as tar:
    tar.extractall('./')

# Load the pre-trained SSD model
model_path = './' + model_file
with tf.io.gfile.GFile(model_path, 'rb') as f:
    graph_def = tf.compat.v1.GraphDef()
    graph_def.ParseFromString(f.read())

In [ ]:
with tf.compat.v1.Session() as sess:
    sess.graph.as_default()
    tf.import_graph_def(graph_def, name='')

    # Get the input and output tensors for the model
    input_tensor = sess.graph.get_tensor_by_name('image_tensor:0')
    detection_boxes = sess.graph.get_tensor_by_name('detection_boxes:0')
    detection_scores = sess.graph.get_tensor_by_name('detection_scores:0')
    detection_classes = sess.graph.get_tensor_by_name('detection_classes:0')
    num_detections = sess.graph.get_tensor_by_name('num_detections:0')

    # Set up the webcam
    cap = cv2.VideoCapture(0)

    while (cap.isOpened()): # while True (prior)
        # Capture a frame from the webcam
        ret, frame = cap.read()

        if frame is None:
            continue

        # Prepare the frame for input to the model
        frame_expanded = np.expand_dims(frame, axis=0)

        # Perform object detection on the frame
        (boxes, scores, classes, num) = sess.run(
            [detection_boxes, detection_scores, detection_classes, num_detections],
            feed_dict={input_tensor: frame_expanded})

        # Draw bounding boxes around the detected objects
        for i in range(int(num[0])):
            score = scores[0][i]
            if score < 0.5:
                continue
            class_name = classes[0][i]
            box = boxes[0][i]
            ymin, xmin, ymax, xmax = box
            x, y, w, h = int(xmin * frame.shape[1]), int(ymin * frame.shape[0]), \
                         int((xmax - xmin) * frame.shape[1]), int((ymax - ymin) * frame.shape[0])
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f'{class_name} ({score:.2f})', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        # Show the frame with the detections
        cv2.imshow('Object Detection', frame)

        # Check for exit key
        if cv2.waitKey(1) == ord('q'):
            break

    # Release the webcam and close
    cap.release()
    cv2.destroyAllWindows

# Finding performance of test set

In [ ]:
#!pip install yolov3-tf2

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os
from yolov3_tf2.models import YoloV3
from keras.layers import Flatten

# Set up paths and parameters
train_dir = '/Users/macbookair/Desktop/541_model/Images_for_train_test/train' # "/content/Training set"
test_dir = '/Users/macbookair/Desktop/541_model/Images_for_train_test/val' # "/content/test set"
img_height, img_width = 416, 416
batch_size = 32
epochs = 10

# Load the training and testing datasets using the ImageDataGenerator class in Keras
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='categorical')

# Load the pre-trained model (e.g. YOLOv3)
model = YoloV3()

# Freeze the convolutional layers to avoid overfitting
for layer in model.layers:
    if "conv" in layer.name:
        layer.trainable = False

# Replace the output layer to match the number of classes in your dataset
num_classes = len(train_generator.class_indices)
output_tensor = model.output[0]  # extract the first tensor from the list
output_layer = tf.keras.layers.Dense(num_classes, activation='softmax')(output_tensor)
model = tf.keras.models.Model(model.input, output_layer)


# Compile the model with an appropriate loss function, optimizer and metrics
#model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
#history = model.fit(
#        train_generator,
#        epochs=epochs,
#        validation_data=test_generator)

# Evaluate the model on the testing set
#score = model.evaluate(test_generator, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])

# Plot the accuracy and loss over time during training
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.title('Model accuracy')
#plt.ylabel('Accuracy')
#plt.xlabel('Epoch')
#plt.legend(['Train', 'Test'], loc='upper left')
#plt.show()

#plt.plot(history.history['loss'])
#plt.plot(history.history['val_loss'])
#plt.title('Model loss')
#plt.ylabel('Loss')
#plt.xlabel('Epoch')
#plt.legend(['Train', 'Test'], loc='upper left')
#plt.show()

Found 2205 images belonging to 1 classes.
Found 552 images belonging to 1 classes.


ValueError: Layer "dense_3" expects 1 input(s), but it received 4 input tensors. Inputs received: [<tf.Tensor 'Placeholder:0' shape=(None, 100, 4) dtype=float32>, <tf.Tensor 'Placeholder_1:0' shape=(None, 100) dtype=float32>, <tf.Tensor 'Placeholder_2:0' shape=(None, 100) dtype=float32>, <tf.Tensor 'Placeholder_3:0' shape=(None,) dtype=int32>]

# Card Detection

In [ ]:
import cv2
import numpy as np
import tensorflow as tf


# Load the trained model
model = tf.keras.models.load_model('card_detection_model.h5')

# Define the classes of objects that the model can detect
classes = ['Ace', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'Jack', 'Queen', 'King']

# Set up the video capture from the webcam
cap = cv2.VideoCapture(0)

# Set up the video writer to save the output video
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('output.avi', fourcc, 20.0, (640, 480))

while(True):
    # Capture a frame from the webcam
    ret, frame = cap.read()

    # Preprocess the image for the model
    img = cv2.resize(frame, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = img / 255.0

    # Run the model inference
    prediction = model.predict(img)[0]

    # Find the class with the highest probability
    class_index = np.argmax(prediction)
    class_name = classes[class_index]

    # Draw a bounding box around the detected card
    cv2.rectangle(frame, (50, 50), (200, 250), (0, 255, 0), 2)
    cv2.putText(frame, class_name, (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Show the frame on the screen
    cv2.imshow('frame', frame)

    # Write the frame to the output video
    out.write(frame)

    # Stop the program if the 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture and writer
cap.release()
out.release()

# Close all windows
cv2.destroyAllWindows()

# Using webcam for real time simulation

In [ ]:
import cv2
import tkinter as tk
from PIL import Image, ImageTk

# Create a GUI window
root = tk.Tk()
root.title("Playing Card Detection")

# Set up a canvas for displaying the webcam feed
canvas = tk.Canvas(root, width=640, height=480)
canvas.pack()

# Load the trained model
model = tf.keras.models.load_model('card_detection_model.h5') # ... # Your trained model here

# Define the function for processing the webcam feed and detecting playing cards
def detect_cards():
    cap = cv2.VideoCapture(0)
    while True:
        ret, frame = cap.read()

        # Process the image frame using the trained model to detect playing cards
        detected_cards = model.detect(frame)

        # Draw rectangles around the detected cards on the frame
        for card in detected_cards:
            cv2.rectangle(frame, (card.xmin, card.ymin), (card.xmax, card.ymax), (0, 255, 0), 2)

        # Convert the image frame to a format that can be displayed in the GUI
        img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(img)
        img = ImageTk.PhotoImage(image=img)

        # Update the canvas with the new image
        canvas.create_image(0, 0, anchor=tk.NW, image=img)
        canvas.update()

        # Exit the loop if the user presses the "q" key
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and destroy the GUI window
    cap.release()
    cv2.destroyAllWindows()

# Call the detect_cards function to start processing the webcam feed
detect_cards()

# Run the GUI loop
root.mainloop()

TclError: ignored